<a href="https://colab.research.google.com/github/jhlee93/AFCompetition/blob/main/2367/Guideline_Submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 안녕하세요. AIFactory입니다.
이 노트북은 제출을 위한 가이드라인 노트북입니다. AIF의 **추론 자동화**는 참가자분들이 작성해주신 이 노트북을 받아 평가 데이터를 추론하고 채점을 하게됩니다.   
  
이번 대회는 스코어 경쟁을 통해 입상자를 선별하는 대회가 아닌,  
AIF의 **추론 자동화**를 경험해보고 이 과정에서 불편한 점, 개선 되었으면 하는 점 등 여러분들의 다양한 의견을 듣기 위해서이니 다양한 의견을 공유해 주시면 감사드리겠습니다.

### 주의 사항
1. 추론 코드와 학습 코드는 반드시 **별도 스크립트**에서 진행해주세요.
2. Colab 노트북이 아닌 경우, 제출 스크립트의 파일 이름은 반드시 ***task.ipynb***로 해주세요.
3. 현재 작업중인 경로에서 모델 추론에 불필요한 파일이 제출되지 않도록 주의해주세요.
4. 모든 경로는 현재 작업중인 스크립트를 기준으로 **상대 경로**로 설정해주세요.
5. AIF의 추론 환경은 Colab과 완전히 동일하지 않으니, 추론 환경에 기 설치된 패키지 항목을 참고하신 후 필요한 패키지를 설치해주세요.

추론 환경에 설치된 라이브러리 리스트
```
tensorflow==2.11.1
keras-cv
torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1
albumentations
transformers
scikit-learn
numpy
Cython
pycocotools
pandas
jupyter
notebook
matplotlib
seaborn
plotly
Pillow
opencv-python
scikit-image
```

# 필요 패키지 설치

In [ ]:
!pip install -U aifactory
!pip install -U gdown

In [ ]:
import os
import random
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ???

In [ ]:
classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

# 모델 로드 함수를 정의해주세요.

- 함수 명칭은 자유롭게 정의해주세요.
- 이 함수가 받는 인자(Arguments)는 없거나, default 값으로 설정 되어야 합니다.
- 모델 로드 함수는 참가자 분들께서 이미 학습 완료한 모델 가중치 파일을 읽어와 메모리에 올려 놓은 후 반환 되어야 합니다.

```python
# Example 1
def tf_load_model():
    ...
    model.load_weights('./weights_dir/my_weights.h5')
    return model

# Example 2
def torch_load_model(weights_path='./my_weights.h5'):
    ...
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model
```


In [ ]:
def load_model():
    # 모델 아키텍쳐
    model = Sequential([
        # ...
        layers.Dense(len(classes))
    ])
    # 모델 가중치 로드
    model.load_weights('./my_weights.h5')

    return model

# 예측 함수를 정의해주세요.
- 예측 함수는 반드시 **2개의 인자**를 다음과 같은 순서대로 받아야합니다.
    - 첫 번째 인자 - AIF가 제공하는 평가 입력 데이터(X_test)
    - 두 번째 인자 - 가중치를 읽어온 모델
- 첫 번째 인자(X_test)는 대회 마다 어떤 형식으로 제공되는지 참가자분들께 공지됩니다.
    - 1분야 대회의 경우 평가용 이미지 파일들의 경로가 담긴 list를 전달합니다.  
    `X_test = ['image_path_000.jpg', 'image_path_001.jpg', ...]`
    - X_test가 제공되는 형식을 참고하여 예측 및 전/후 처리를 통해 **제출 규칙**에 맞게 결과를 반환해주세요.
- **제출 규칙** 또한 대회 마다 규칙이 정해져있습니다.
    - 1분야 대회의 경우 X_test 전체의 예측값을 다음과 같이 list 형태로 담아 반환해주세요.  
    `['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips', ... ]`
- 예측 함수가 제출 규칙에 맞게 반환되면 AIF는 정답 데이터와 비교하여 평가 점수를 계산합니다.  
다음 코드를 참고하여 모델 로드 함수와 예측 함수를 정의해주세요.
    ```python
    # AIF 채점 코드 예시
    X_test = ['image_path_000.jpg', 'image_path_001.jpg', ...]
    y_test = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips', ... ]

    model = tf_load_model() # 참가팀 모델 로드 함수 실행
    y_pred = predict(X_test, model) # 참가팀 예측 함수 실행

    from sklearn.metrics import f1_score
    score = f1_score(y_test, y_pred, average='macro')
    ```


In [ ]:
# 예측 함수는 AIF에서 제공하는 X_test의 입력 정보와, 메모리에 올라간 사전 학습 모델을 입력 받습니다.
def predict(X_test, model):
    my_result = []
    for X_test_path in X_test:
        image = Image.open(X_test_path).resize((180,180)) # 이미지 읽어오기

        # 입력 형식 변환
        image = np.array(image, np.uint8) # (180, 180, 3)
        image = np.expand_dims(image, axis=0) # (1, 180, 180, 3)
        
        # 제출 형식에 맞게 변환
        pred_proba = model.predict(image, verbose=False)
        pred_id = np.argmax(pred_proba, axis=1)[0]
        pred_name = classes[pred_id]

        my_result.append(pred_name)

    return my_result


# submit 함수를 정의해 주세요.
- 이 함수의 명칭은 반드시 **submit** 으로 해주세요.
- **submit** 함수는 AIF가 참가자분들이 정의한 2개의 함수를 전달받아 실행하기 위해 사용됩니다.
- **submit** 함수를 통해 위에서정의한 함수를 **1.모델 로드 함수**와 **2.예측 함수** 순으로 반환해 주세요.

In [ ]:

def submit():
    return load_model, predict

# 제출하기
- aifactory 라이브러리를 통해 submit 함수를 제출합니다.
- aif.submit 함수는 반드시 `if __name__ == "__main__":` 아래에 작성해주세요.
- aifactory.score.submit 함수는 3개의 인자를 받습니다.
    - ***model_name*** - 제출시 리더보드에 기록하고자 하는 문구
    - ***key*** - 대회 참가자에게 제공되는 고유 key
    - ***func*** - submit 함수

In [ ]:
import aifactory.score as aif

if __name__ == "__main__":
    aif.submit(model_name="MyFirstSubmission",
               key='TASK_KEY',
               func=submit
               )